In [1]:
import os
import argparse
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torch.nn.functional as F
import sklearn
from torch.autograd import Variable
from sklearn.preprocessing import normalize
from itertools import *
import matplotlib.pyplot as plt

# Parameters

In [2]:
NGF = 5 # Number of filters in first layer of generator
NDF = 5 # Number of filters in first layer of discriminator
POOL_SIZE = 50 # pool_size
IMG_CHANNEL = 1 # RGB format （但我們的黑白用1）
LR=0.005
BATCH_SIZE = 100#一個月 4 * 5
                #半年 6 * 4 * 5 = 120
OUTPUT_SIZE = 30
INPUT_SIZE = OUTPUT_SIZE
input_nc = 1
output_nc = 1

BETA1 = 0.5

# Preprocessing

In [4]:
from os import listdir
from os.path import isfile, join

import xlrd
disk = './'

mypath = disk+'quantity'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]



def readFromFiles(files):
    data = []
    mypath = disk+'quantity'
    for i in range(0,len(files)):
        path = mypath+'/'+files[i]
        date = files[i][2:len(files[i])-4]
        #print(date)
        wb = xlrd.open_workbook(path)
        ws = wb.sheets()[0]

          
        for j in range(0,ws.nrows):
            code = ws.cell(j,1).value
            #print(code)
            if str(code)[0:4] == str(2330):
                quantity = ws.cell(j,3).value
                data.append([date,quantity])
                #print(quantity)
                break
        
    return data

   
issued_shares = readFromFiles(onlyfiles)
issued_shares[1] #每年每個月總發行股數

['200708', 26420379.0]

In [5]:
def SpecialSplit(line):
    cannotSplit = False
    splitIndexs = []
    #print(line)
    for index in range(0,len(line)):
        
        c = line[index]
        #print(c)
        if c == '"':
            cannotSplit = not cannotSplit
            continue
        if c ==',' and cannotSplit == False:
            splitIndexs.append(index)
            #print('i catch ,')
    #print(splitIndexs)
    startIndex = 0
    splitLine = []
    for index in splitIndexs:
        splitLine.append(line[startIndex:index])
        #print('???')
        startIndex=index+1
    splitLine.append(line[startIndex:len(line)-1])
    splitLine2 = []
    for l in splitLine:
        s = ''
        for c in l:
            if c !='"':
                s += c
        splitLine2.append(s)
    return splitLine2

In [6]:
#股票資料2007-2018            
#reader =  open(disk+'半導體股票資料.csv','r',encoding='BIG5')
reader =  open(disk+'股票資料2007-2018.csv','r',encoding='BIG5')
colName = reader.readline()
dates= []
rows = []
def fitNumber(s):
    for c in s:
        if c not in [str(f) for f in range(0,10)]:
            return False
    return True
check = False
for line in reader.readlines():
    lines = SpecialSplit (line) #def SpecialSplit
    #print(line)
    #print(lines)
    if len(lines[0])==4 and len(lines[1])<=2 and len(lines[2])<=2 and fitNumber(lines[0]) and fitNumber(lines[1]) and fitNumber(lines[2]):
        if len(lines[1])==1 :
            lines[1] = '0'+lines[1]
            
            
            
        if len(lines[2]) == 1:
            lines[2] = '0'+lines[2]
            
        
        
        check = True    
        date = (lines[0]+lines[1]+lines[2])
        #print(date)
    if lines[0] == '2330':
        if check:
            #print(date)
            check = False
            dates.append(date)
            rows.append(lines)
    
# print(rows)
# ##3 筆數
# ##8 收盤價
# ##5開盤價
Quantity = [ line[3]for line in rows]
Price = [line[5] for line in rows]
Quantity= [ float((f.replace('"','')).replace(',','')) for f in Quantity]
Price = [ float(f) for f in Price]


# Normalization + Minmax

In [7]:
def Normarlization(data):
    data2 = []
    for i in range(0,len(data)-1):
        today = data[i]
        tomorrow = data[i+1]
        
        val =np.log(tomorrow) - np.log(today)
        #val = tomorrow / today
        data2.append(val)
       
    return data2

Nor_Quantity = Normarlization(Quantity)
Nor_Price= Normarlization(Price)
a = len(issued_shares)
b = len(Nor_Price)
c = len(Price)
d = len(Nor_Quantity)
e = len(Quantity)
print('length of issued_shares: {a}'.format(a=a))
print('length of Quantity: {e}'.format(e=e))
print('lenght of Price: {c}'.format(c=c))
print('lenght of Nor_Quantity: {d}'.format(d=d))
print('lenght of Nor_Price: {b}'.format(b=b))

length of issued_shares: 138
length of Quantity: 2948
lenght of Price: 2948
lenght of Nor_Quantity: 2947
lenght of Nor_Price: 2947


In [8]:
norPriceNumpy= np.asarray(Nor_Price, dtype=np.float32)
norQuantityNumpy= np.asarray(Nor_Quantity, dtype=np.float32)

type(norPriceNumpy)

numpy.ndarray

In [9]:
#Data is between 0 and 1
def MinMaxNormalization(data): 
    assert np.array(data).ndim == 1 
    data = [ (f-np.min(data) )/ (np.max(data)-np.min(data) ) for f in data]
    return data

MinMax_Quantity = MinMaxNormalization(norQuantityNumpy)
MinMax_Price = MinMaxNormalization(norPriceNumpy)

type(MinMax_Quantity)
len(MinMax_Quantity)
for i in range (22):
    MinMax_Quantity.pop()
    MinMax_Price.pop()

In [10]:
# 開始切成二維，input為5*5，共117列
print(len(MinMax_Price))
dataRow = 5
dataCol = 5
dataBatch = 117
print(dataRow,dataCol,dataBatch)

2925
5 5 117


In [11]:
#list轉nparray，在用np 1d->2d
MinMax_Quantity = np.asarray(MinMax_Quantity, dtype=np.float32)
MinMax_Price = np.asarray(MinMax_Price, dtype=np.float32)

Quantity = np.array(MinMax_Quantity).reshape(dataBatch,dataCol,dataRow)
Price = np.array(MinMax_Price).reshape(dataBatch,dataCol,dataRow)

In [12]:
print(Quantity)

[[[0.59764415 0.21551003 0.5397831  0.42165044 0.40774533]
  [0.7726673  0.44470325 0.7405554  0.10807862 0.59313816]
  [0.31152695 0.41907516 0.38780972 0.48513412 0.31308347]
  [0.6422431  0.25838876 0.79057705 0.07426015 0.40898535]
  [0.6982764  0.15928875 0.4577276  0.35601893 0.5315501 ]]

 [[0.43287444 0.4985944  0.45307726 0.5788343  0.52574944]
  [0.17125197 0.73257065 0.41589087 0.5968069  0.1683875 ]
  [0.7204234  0.3326046  0.22925366 0.39333466 0.48352784]
  [0.44766426 0.7770109  0.16709553 0.4724675  0.32110977]
  [0.3045189  0.5585867  0.53139836 0.65211165 0.24322768]]

 [[0.28901517 0.5880452  0.4313202  0.39338687 0.48584312]
  [0.32961893 0.46284822 0.6693102  0.24040927 0.47389007]
  [0.47015858 0.4052575  0.3747289  0.50759596 0.4804104 ]
  [0.48542085 0.69323754 0.13677691 0.5966256  0.5568577 ]
  [0.1274281  0.6513594  0.29467174 0.50205505 0.48510695]]

 ...

 [[0.3463892  0.5096103  0.46365398 0.32742053 0.44353223]
  [0.37873578 0.4148677  0.48792908 0.409462

In [13]:
Quantity = torch.Tensor(Quantity)
Price = torch.Tensor(Price)

In [14]:
Price.size()

torch.Size([117, 5, 5])

In [15]:
print(torch.unsqueeze(Quantity,1).size()) #(batch,channel,height,width)

torch.Size([117, 1, 5, 5])


In [16]:
Quantity = torch.unsqueeze(Quantity,1) #(117, 1, 5, 5)
Price = torch.unsqueeze(Price,1) #(117, 1, 5, 5)

In [17]:
Price.size()

torch.Size([117, 1, 5, 5])

In [18]:
Price = Variable(Price , requires_grad = True)
Quantity = Variable(Quantity , requires_grad = True)

# Generator
### Step1. Encoding
從領域A的圖拆出成特徵。
### Step2. Transformation
將領域A的圖的特徵轉換成領域B的圖的特徵。
### Step3. Decoding
以這些特徵拼出一張領域B的圖。

In [20]:
class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()
        self.conv_block = nn.Sequential(
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features),
            nn.ReLU(inplace=True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features) 
        )

    def forward(self, x):
        return x + self.conv_block(x)

#----------------------------------------

in_features = 5
out_features = 10
class Generator(nn.Module):
    def __init__(self, input_nc, output_nc, n_residual_blocks=9):
        super(Generator, self).__init__()
        # Initial convolution block       
        self.encoding = nn.Sequential(
            nn.ReflectionPad2d(3),
            nn.Conv2d(input_nc, 5, 3),
            nn.InstanceNorm2d(5),
            nn.ReLU(inplace=True), 
            # Downsampling
            nn.Conv2d(in_features, out_features, 3, stride=2, padding=1),
            nn.InstanceNorm2d(out_features),
            nn.ReLU(inplace=True) ,
            nn.Conv2d(out_features, out_features*2, 3, stride=2, padding=1),
            nn.InstanceNorm2d(out_features*2),
            nn.ReLU(inplace=True) 
        )

        self.transformation = nn.Sequential(
            ResidualBlock(out_features*2),
            ResidualBlock(out_features*2),
            ResidualBlock(out_features*2),
            ResidualBlock(out_features*2),
            ResidualBlock(out_features*2),
            ResidualBlock(out_features*2),
            ResidualBlock(out_features*2),
            ResidualBlock(out_features*2),
            ResidualBlock(out_features*2)
        )
            
        
        self.decoding =nn.Sequential( 
            nn.ConvTranspose2d(out_features*2, out_features, 3, stride=1, padding=1),
            nn.InstanceNorm2d(out_features),
            nn.ReLU(inplace=True), 
            nn.ConvTranspose2d(out_features, in_features, 3, stride=2, padding=1),
            nn.InstanceNorm2d(in_features),
            nn.ReLU(inplace=True) ,
            nn.ReflectionPad2d(1),
            nn.Conv2d(5, output_nc, 3),
            nn.Tanh()
        )
    def forward(self, input):
        encodingOutput = self.encoding(input)
        transformationOutput = self.transformation(encodingOutput)
        decodingOutput = self.decoding(transformationOutput)
        return decodingOutput
netG_P2Q = Generator(1, 1)
netG_Q2P = Generator(1, 1)

In [21]:
# class Generator(nn.Module):
#     def __init__(self):
#         super(Generator,self).__init__()
        
#         #從領域A的圖拆出成特徵。
#         self.encoding = nn.Sequential(  
#             # input is 1 x 64 x 64
#             # Conv2d(inputChannel,outChannel,kernelSize,stride,padding)
#             nn.Conv2d(IMG_CHANNEL, NGF, 1, 2, 0, bias=False),
#             nn.LeakyReLU(0.2, inplace=True),
            
#             nn.Conv2d(NGF, NGF*2, 1, 2, 0, bias=False),
#             nn.LeakyReLU(0.2, inplace=True),
            
#             nn.Conv2d(NGF*2, NGF*4, 1, 2, 0, bias=False),
#             nn.LeakyReLU(0.2, inplace=True),
#         )
        
#         #將領域A的圖的特徵轉換成領域B的圖的特徵。
#         self.transformation = nn.Sequential(
#             ResNetBasicBlock(IMG_CHANNEL*1, IMG_CHANNEL*1),
#             ResNetBasicBlock(IMG_CHANNEL*1, IMG_CHANNEL*1),
#             ResNetBasicBlock(IMG_CHANNEL*1, IMG_CHANNEL*1),
#             ResNetBasicBlock(IMG_CHANNEL*1, IMG_CHANNEL*1),
#             ResNetBasicBlock(IMG_CHANNEL*1, IMG_CHANNEL*1),
#             ResNetBasicBlock(IMG_CHANNEL*1, IMG_CHANNEL*1),
#             ResNetBasicBlock(IMG_CHANNEL*1, IMG_CHANNEL*1),
#             ResNetBasicBlock(IMG_CHANNEL*1, IMG_CHANNEL*1),
#         )
        
#         #以這些特徵拼出一張領域B的圖。
#         self.decoding =nn.Sequential(
#             nn.ConvTranspose2d(1, NGF*8, 1, 2, 0, bias=False),
#             nn.ConvTranspose2d(NGF*8, NGF*4, 1, 2, 0, bias=False),
#             nn.ConvTranspose2d(NGF*4, NGF*2, 1, 2, 0, bias=False),
#             nn.ConvTranspose2d(NGF*2, NGF, 1, 2, 0, bias=False),
#             nn.ConvTranspose2d(NGF, IMG_CHANNEL, 1, 2, 0, bias=False)
            
#         )
        
#     def forward(self, input):
#         encodingOutput = self.encoding(input)
#         transformationOutput = self.transformation(encodingOutput)
# #         decodingOutput = self.decoding(transformationOutput)
# #         decodingOutput = self.decoding(encodingOutput)
#         return transformationOutput.view(-1, 1) .squeeze(1)
# netG = Generator()

# Discriminator

In [22]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential (
            # input is (nc) x 64 x 64
            nn.Conv2d(IMG_CHANNEL, NDF, 1, 2, 0, bias=False),
            nn. LeakyReLU (0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(NDF, NDF * 2, 1, 2, 0, bias=False),
            nn.BatchNorm2d(NDF * 2),
            nn. LeakyReLU (0.2, inplace=True),
            # state size.(ndf*2)x16x16
            nn.Conv2d(NDF * 2, NDF * 4, 1, 2, 0, bias=False),
            nn.BatchNorm2d(NDF * 4),
            nn. LeakyReLU (0.2, inplace=True),
            # state size. (ndf*4)x8x8
            nn.Conv2d(NDF * 4, NDF * 8, 1, 2, 0, bias=False),
            nn.BatchNorm2d(NDF * 8),
            nn . LeakyReLU (0.2, inplace=True),
            # state size. (ndf*8) x4 x 4
            nn.Conv2d(NDF * 8, 1, 1, 1, 0, bias=False),
            nn.Sigmoid()
        ) 
            
    def forward(self, input):
        output = self.main(input)
        return output.view(-1, 1) .squeeze(1)
     
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)    

netD_P = Discriminator()
netD_Q = Discriminator()

# Build Model

In [23]:
# #a = Price, b = Quantity
# genA2B=netG(Price) #將a的真實資料丟入Genrator中，產生出假 b
# genB2A=netG(Quantity) #將b的真實資料丟入Genrator中，產生出假 a

# desA=netD(Price) #將a的真實資料丟入discriminator中，讓他知道是真 a(label real)
# desB=netD(Quantity) #將b的真實資料丟入discriminator中，讓他知道是真 b(label real)

# desFakeA=netD(genB2A) # 將b生出的假a資料，丟回Discriminator訓練 (label fake)
# desFakeB=netD(genA2B) # 將a生出的假b資料，丟回Discriminator訓練 (label fake)

# genFakeB2A=netG(genA2B) # 將由b產生的假a，丟入generator產生b
# genFakeA2B=netG(genB2A) # 將由a產生的假b，丟入generator產生a
# genA2B.parameters()

# Loss Function & Optimizer

In [24]:
MSELoss = nn.MSELoss()
MAELoss = nn.L1Loss()

# #MSE(Discriminator)
# lossRealA = MSELoss(desA,1)
# lossRealB = MSELoss(desB,1)
# lossFakeA = MSELoss(desFakeA,0)
# lossFakeB = MSELoss(desFakeB,0)
# lossDisA = (lossRealA + lossFakeA)/2
# lossDisB = (lossRealB + lossFakeB)/2

# #MSE(Generator)
# lossRealA = MSELoss(desFakeA,1)
# lossRealB = MSELoss(desFakeB,1)

# #cyclic
# L1loss = torch.nn.L1Loss() 
# cyc_loss =L1loss((input_A-genFakeB2A)) + L1Loss((input_B-genFakeA2B))

criterion_GAN = torch.nn.MSELoss()
criterion_cycle = torch.nn.L1Loss()
criterion_identity = torch.nn.L1Loss()

### optimizer

In [29]:
import itertools
# optGenA2B=optim.Adam(genA2B.parameters(),LR)
# optGenB2A=optim.Adam(genB2A.parameters(),LR)
# optDesA=optim.Adam(desA.parameters(),LR)
# optDesB=optim.Adam(desB.parameters(),LR)
optimizer_G = torch.optim.Adam(itertools.chain(netG_P2Q.parameters(), netG_Q2P.parameters()),
                                lr=LR, betas=(0.5, 0.999))
optimizer_D_P = torch.optim.Adam(netD_P.parameters(), lr=LR, betas=(0.5, 0.999))
optimizer_D_Q = torch.optim.Adam(netD_Q.parameters(), lr=LR, betas=(0.5, 0.999))

### Training

In [30]:
real_Price = Variable(Price , requires_grad = True)
real_Quantity = Variable(Quantity , requires_grad = True)

In [32]:
#Discriminator Q->REAL P->FAKE

EPOCHS = 3
device = torch.device("cpu")
real_label = 1
fake_label = 0
for epoch in range(EPOCHS):
    for i, data in enumerate(Quantity, 0):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        # train with real
        netD_Q.zero_grad()
        real_cpu = data[0].to(device)
        batch_size = 1
        label = torch.full((batch_size,), real_label,
                           dtype=real_cpu.dtype, device=device)

        output = netD_Q(Quantity) #resnet會影響
        errD_real = MSELoss(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        # train with fake
        fake = netG_P2Q(Price)
        print(fake) #price --> quantity(fake)
        label.fill_(fake_label)
        output = netD_Q(fake.detach())
        errD_fake = MSELoss(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizer_D_Q.step()

tensor([[[[-0.5659, -0.6364, -0.1264, -0.1151, -0.4337],
          [-0.5536,  0.4305, -0.5178, -0.2338,  0.0831],
          [-0.1312, -0.1972,  0.0817, -0.6462, -0.0365],
          [-0.4844, -0.4191, -0.2185, -0.4317, -0.3608],
          [-0.3658, -0.2686, -0.6794, -0.2619, -0.6421]]],


        [[[-0.4924, -0.4682, -0.6241, -0.2627, -0.7276],
          [-0.7819,  0.2739, -0.4130, -0.0202, -0.7235],
          [-0.0863, -0.0637, -0.8215, -0.5667, -0.6475],
          [ 0.0316, -0.1190,  0.2924,  0.2162, -0.5385],
          [-0.4812, -0.1020, -0.7878, -0.3078, -0.7449]]],


        [[[-0.1078, -0.4803, -0.1916, -0.3077, -0.4077],
          [-0.5039, -0.2084, -0.1930, -0.3063, -0.0517],
          [-0.7782, -0.5832, -0.8440, -0.5989, -0.8245],
          [-0.5788,  0.1995, -0.4076,  0.2637, -0.4316],
          [-0.8292, -0.4305, -0.5672, -0.1839, -0.6568]]],


        ...,


        [[[-0.5804, -0.2060, -0.5789, -0.5456, -0.4387],
          [-0.7601, -0.6140, -0.4412, -0.0437, -0.5146],
    

tensor([[[[-0.5659, -0.6364, -0.1264, -0.1151, -0.4337],
          [-0.5536,  0.4305, -0.5178, -0.2338,  0.0831],
          [-0.1312, -0.1972,  0.0817, -0.6462, -0.0365],
          [-0.4844, -0.4191, -0.2185, -0.4317, -0.3608],
          [-0.3658, -0.2686, -0.6794, -0.2619, -0.6421]]],


        [[[-0.4924, -0.4682, -0.6241, -0.2627, -0.7276],
          [-0.7819,  0.2739, -0.4130, -0.0202, -0.7235],
          [-0.0863, -0.0637, -0.8215, -0.5667, -0.6475],
          [ 0.0316, -0.1190,  0.2924,  0.2162, -0.5385],
          [-0.4812, -0.1020, -0.7878, -0.3078, -0.7449]]],


        [[[-0.1078, -0.4803, -0.1916, -0.3077, -0.4077],
          [-0.5039, -0.2084, -0.1930, -0.3063, -0.0517],
          [-0.7782, -0.5832, -0.8440, -0.5989, -0.8245],
          [-0.5788,  0.1995, -0.4076,  0.2637, -0.4316],
          [-0.8292, -0.4305, -0.5672, -0.1839, -0.6568]]],


        ...,


        [[[-0.5804, -0.2060, -0.5789, -0.5456, -0.4387],
          [-0.7601, -0.6140, -0.4412, -0.0437, -0.5146],
    

tensor([[[[-0.5659, -0.6364, -0.1264, -0.1151, -0.4337],
          [-0.5536,  0.4305, -0.5178, -0.2338,  0.0831],
          [-0.1312, -0.1972,  0.0817, -0.6462, -0.0365],
          [-0.4844, -0.4191, -0.2185, -0.4317, -0.3608],
          [-0.3658, -0.2686, -0.6794, -0.2619, -0.6421]]],


        [[[-0.4924, -0.4682, -0.6241, -0.2627, -0.7276],
          [-0.7819,  0.2739, -0.4130, -0.0202, -0.7235],
          [-0.0863, -0.0637, -0.8215, -0.5667, -0.6475],
          [ 0.0316, -0.1190,  0.2924,  0.2162, -0.5385],
          [-0.4812, -0.1020, -0.7878, -0.3078, -0.7449]]],


        [[[-0.1078, -0.4803, -0.1916, -0.3077, -0.4077],
          [-0.5039, -0.2084, -0.1930, -0.3063, -0.0517],
          [-0.7782, -0.5832, -0.8440, -0.5989, -0.8245],
          [-0.5788,  0.1995, -0.4076,  0.2637, -0.4316],
          [-0.8292, -0.4305, -0.5672, -0.1839, -0.6568]]],


        ...,


        [[[-0.5804, -0.2060, -0.5789, -0.5456, -0.4387],
          [-0.7601, -0.6140, -0.4412, -0.0437, -0.5146],
    

tensor([[[[-0.5659, -0.6364, -0.1264, -0.1151, -0.4337],
          [-0.5536,  0.4305, -0.5178, -0.2338,  0.0831],
          [-0.1312, -0.1972,  0.0817, -0.6462, -0.0365],
          [-0.4844, -0.4191, -0.2185, -0.4317, -0.3608],
          [-0.3658, -0.2686, -0.6794, -0.2619, -0.6421]]],


        [[[-0.4924, -0.4682, -0.6241, -0.2627, -0.7276],
          [-0.7819,  0.2739, -0.4130, -0.0202, -0.7235],
          [-0.0863, -0.0637, -0.8215, -0.5667, -0.6475],
          [ 0.0316, -0.1190,  0.2924,  0.2162, -0.5385],
          [-0.4812, -0.1020, -0.7878, -0.3078, -0.7449]]],


        [[[-0.1078, -0.4803, -0.1916, -0.3077, -0.4077],
          [-0.5039, -0.2084, -0.1930, -0.3063, -0.0517],
          [-0.7782, -0.5832, -0.8440, -0.5989, -0.8245],
          [-0.5788,  0.1995, -0.4076,  0.2637, -0.4316],
          [-0.8292, -0.4305, -0.5672, -0.1839, -0.6568]]],


        ...,


        [[[-0.5804, -0.2060, -0.5789, -0.5456, -0.4387],
          [-0.7601, -0.6140, -0.4412, -0.0437, -0.5146],
    

tensor([[[[-0.5659, -0.6364, -0.1264, -0.1151, -0.4337],
          [-0.5536,  0.4305, -0.5178, -0.2338,  0.0831],
          [-0.1312, -0.1972,  0.0817, -0.6462, -0.0365],
          [-0.4844, -0.4191, -0.2185, -0.4317, -0.3608],
          [-0.3658, -0.2686, -0.6794, -0.2619, -0.6421]]],


        [[[-0.4924, -0.4682, -0.6241, -0.2627, -0.7276],
          [-0.7819,  0.2739, -0.4130, -0.0202, -0.7235],
          [-0.0863, -0.0637, -0.8215, -0.5667, -0.6475],
          [ 0.0316, -0.1190,  0.2924,  0.2162, -0.5385],
          [-0.4812, -0.1020, -0.7878, -0.3078, -0.7449]]],


        [[[-0.1078, -0.4803, -0.1916, -0.3077, -0.4077],
          [-0.5039, -0.2084, -0.1930, -0.3063, -0.0517],
          [-0.7782, -0.5832, -0.8440, -0.5989, -0.8245],
          [-0.5788,  0.1995, -0.4076,  0.2637, -0.4316],
          [-0.8292, -0.4305, -0.5672, -0.1839, -0.6568]]],


        ...,


        [[[-0.5804, -0.2060, -0.5789, -0.5456, -0.4387],
          [-0.7601, -0.6140, -0.4412, -0.0437, -0.5146],
    

...

tensor([[[[-0.5659, -0.6364, -0.1264, -0.1151, -0.4337],
          [-0.5536,  0.4305, -0.5178, -0.2338,  0.0831],
          [-0.1312, -0.1972,  0.0817, -0.6462, -0.0365],
          [-0.4844, -0.4191, -0.2185, -0.4317, -0.3608],
          [-0.3658, -0.2686, -0.6794, -0.2619, -0.6421]]],


        [[[-0.4924, -0.4682, -0.6241, -0.2627, -0.7276],
          [-0.7819,  0.2739, -0.4130, -0.0202, -0.7235],
          [-0.0863, -0.0637, -0.8215, -0.5667, -0.6475],
          [ 0.0316, -0.1190,  0.2924,  0.2162, -0.5385],
          [-0.4812, -0.1020, -0.7878, -0.3078, -0.7449]]],


        [[[-0.1078, -0.4803, -0.1916, -0.3077, -0.4077],
          [-0.5039, -0.2084, -0.1930, -0.3063, -0.0517],
          [-0.7782, -0.5832, -0.8440, -0.5989, -0.8245],
          [-0.5788,  0.1995, -0.4076,  0.2637, -0.4316],
          [-0.8292, -0.4305, -0.5672, -0.1839, -0.6568]]],


        ...,


        [[[-0.5804, -0.2060, -0.5789, -0.5456, -0.4387],
          [-0.7601, -0.6140, -0.4412, -0.0437, -0.5146],
    

tensor([[[[-0.5659, -0.6364, -0.1264, -0.1151, -0.4337],
          [-0.5536,  0.4305, -0.5178, -0.2338,  0.0831],
          [-0.1312, -0.1972,  0.0817, -0.6462, -0.0365],
          [-0.4844, -0.4191, -0.2185, -0.4317, -0.3608],
          [-0.3658, -0.2686, -0.6794, -0.2619, -0.6421]]],


        [[[-0.4924, -0.4682, -0.6241, -0.2627, -0.7276],
          [-0.7819,  0.2739, -0.4130, -0.0202, -0.7235],
          [-0.0863, -0.0637, -0.8215, -0.5667, -0.6475],
          [ 0.0316, -0.1190,  0.2924,  0.2162, -0.5385],
          [-0.4812, -0.1020, -0.7878, -0.3078, -0.7449]]],


        [[[-0.1078, -0.4803, -0.1916, -0.3077, -0.4077],
          [-0.5039, -0.2084, -0.1930, -0.3063, -0.0517],
          [-0.7782, -0.5832, -0.8440, -0.5989, -0.8245],
          [-0.5788,  0.1995, -0.4076,  0.2637, -0.4316],
          [-0.8292, -0.4305, -0.5672, -0.1839, -0.6568]]],


        ...,


        [[[-0.5804, -0.2060, -0.5789, -0.5456, -0.4387],
          [-0.7601, -0.6140, -0.4412, -0.0437, -0.5146],
    

tensor([[[[-0.5659, -0.6364, -0.1264, -0.1151, -0.4337],
          [-0.5536,  0.4305, -0.5178, -0.2338,  0.0831],
          [-0.1312, -0.1972,  0.0817, -0.6462, -0.0365],
          [-0.4844, -0.4191, -0.2185, -0.4317, -0.3608],
          [-0.3658, -0.2686, -0.6794, -0.2619, -0.6421]]],


        [[[-0.4924, -0.4682, -0.6241, -0.2627, -0.7276],
          [-0.7819,  0.2739, -0.4130, -0.0202, -0.7235],
          [-0.0863, -0.0637, -0.8215, -0.5667, -0.6475],
          [ 0.0316, -0.1190,  0.2924,  0.2162, -0.5385],
          [-0.4812, -0.1020, -0.7878, -0.3078, -0.7449]]],


        [[[-0.1078, -0.4803, -0.1916, -0.3077, -0.4077],
          [-0.5039, -0.2084, -0.1930, -0.3063, -0.0517],
          [-0.7782, -0.5832, -0.8440, -0.5989, -0.8245],
          [-0.5788,  0.1995, -0.4076,  0.2637, -0.4316],
          [-0.8292, -0.4305, -0.5672, -0.1839, -0.6568]]],


        ...,


        [[[-0.5804, -0.2060, -0.5789, -0.5456, -0.4387],
          [-0.7601, -0.6140, -0.4412, -0.0437, -0.5146],
    

tensor([[[[-0.5659, -0.6364, -0.1264, -0.1151, -0.4337],
          [-0.5536,  0.4305, -0.5178, -0.2338,  0.0831],
          [-0.1312, -0.1972,  0.0817, -0.6462, -0.0365],
          [-0.4844, -0.4191, -0.2185, -0.4317, -0.3608],
          [-0.3658, -0.2686, -0.6794, -0.2619, -0.6421]]],


        [[[-0.4924, -0.4682, -0.6241, -0.2627, -0.7276],
          [-0.7819,  0.2739, -0.4130, -0.0202, -0.7235],
          [-0.0863, -0.0637, -0.8215, -0.5667, -0.6475],
          [ 0.0316, -0.1190,  0.2924,  0.2162, -0.5385],
          [-0.4812, -0.1020, -0.7878, -0.3078, -0.7449]]],


        [[[-0.1078, -0.4803, -0.1916, -0.3077, -0.4077],
          [-0.5039, -0.2084, -0.1930, -0.3063, -0.0517],
          [-0.7782, -0.5832, -0.8440, -0.5989, -0.8245],
          [-0.5788,  0.1995, -0.4076,  0.2637, -0.4316],
          [-0.8292, -0.4305, -0.5672, -0.1839, -0.6568]]],


        ...,


        [[[-0.5804, -0.2060, -0.5789, -0.5456, -0.4387],
          [-0.7601, -0.6140, -0.4412, -0.0437, -0.5146],
    

tensor([[[[-0.5659, -0.6364, -0.1264, -0.1151, -0.4337],
          [-0.5536,  0.4305, -0.5178, -0.2338,  0.0831],
          [-0.1312, -0.1972,  0.0817, -0.6462, -0.0365],
          [-0.4844, -0.4191, -0.2185, -0.4317, -0.3608],
          [-0.3658, -0.2686, -0.6794, -0.2619, -0.6421]]],


        [[[-0.4924, -0.4682, -0.6241, -0.2627, -0.7276],
          [-0.7819,  0.2739, -0.4130, -0.0202, -0.7235],
          [-0.0863, -0.0637, -0.8215, -0.5667, -0.6475],
          [ 0.0316, -0.1190,  0.2924,  0.2162, -0.5385],
          [-0.4812, -0.1020, -0.7878, -0.3078, -0.7449]]],


        [[[-0.1078, -0.4803, -0.1916, -0.3077, -0.4077],
          [-0.5039, -0.2084, -0.1930, -0.3063, -0.0517],
          [-0.7782, -0.5832, -0.8440, -0.5989, -0.8245],
          [-0.5788,  0.1995, -0.4076,  0.2637, -0.4316],
          [-0.8292, -0.4305, -0.5672, -0.1839, -0.6568]]],


        ...,


        [[[-0.5804, -0.2060, -0.5789, -0.5456, -0.4387],
          [-0.7601, -0.6140, -0.4412, -0.0437, -0.5146],
    

In [33]:
#Discriminator P->REAL Q->FAKE

EPOCHS = 3
device = torch.device("cpu")
real_label = 1
fake_label = 0
for epoch in range(EPOCHS):
    for i, data in enumerate(Quantity, 0):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        # train with real
        netD_P.zero_grad()
        real_cpu = data[0].to(device)
        batch_size = 1
        label = torch.full((batch_size,), real_label,
                           dtype=real_cpu.dtype, device=device)

        output = netD_P(Price) #resnet會影響
        errD_real = MSELoss(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        # train with fake
        fake = netG_Q2P(Quantity)
        print(fake) # quantity -->price (fake)
        label.fill_(fake_label)
        output = netD_Q(fake.detach())
        errD_fake = MSELoss(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizer_D_P.step()

tensor([[[[-0.0136, -0.2407, -0.1060,  0.5737, -0.1012],
          [ 0.1682, -0.0440,  0.5148,  0.7115,  0.5687],
          [ 0.5914,  0.3073,  0.7647,  0.4631,  0.5266],
          [ 0.5126,  0.3898,  0.5129,  0.5072,  0.3920],
          [ 0.1739,  0.3780, -0.0565,  0.8841,  0.2094]]],


        [[[ 0.2769,  0.2192,  0.2228,  0.2374,  0.0857],
          [ 0.4258,  0.0474,  0.4744,  0.7028,  0.4913],
          [ 0.5954,  0.5196,  0.7530,  0.5446,  0.3527],
          [ 0.4627,  0.1940,  0.5212, -0.1657,  0.4038],
          [ 0.2904,  0.7572,  0.2598,  0.4926,  0.3253]]],


        [[[ 0.4439,  0.5963,  0.5554,  0.3550,  0.1570],
          [ 0.7325,  0.7591,  0.8513,  0.4449,  0.3250],
          [ 0.1793,  0.3053, -0.0891,  0.4469,  0.4883],
          [-0.0622, -0.1304,  0.1926,  0.2591,  0.4404],
          [ 0.0944,  0.3512,  0.3354,  0.3881,  0.5322]]],


        ...,


        [[[ 0.5209,  0.4986,  0.5187, -0.1027,  0.6966],
          [ 0.2983,  0.7806,  0.4102,  0.0292,  0.0701],
    

tensor([[[[-0.0136, -0.2407, -0.1060,  0.5737, -0.1012],
          [ 0.1682, -0.0440,  0.5148,  0.7115,  0.5687],
          [ 0.5914,  0.3073,  0.7647,  0.4631,  0.5266],
          [ 0.5126,  0.3898,  0.5129,  0.5072,  0.3920],
          [ 0.1739,  0.3780, -0.0565,  0.8841,  0.2094]]],


        [[[ 0.2769,  0.2192,  0.2228,  0.2374,  0.0857],
          [ 0.4258,  0.0474,  0.4744,  0.7028,  0.4913],
          [ 0.5954,  0.5196,  0.7530,  0.5446,  0.3527],
          [ 0.4627,  0.1940,  0.5212, -0.1657,  0.4038],
          [ 0.2904,  0.7572,  0.2598,  0.4926,  0.3253]]],


        [[[ 0.4439,  0.5963,  0.5554,  0.3550,  0.1570],
          [ 0.7325,  0.7591,  0.8513,  0.4449,  0.3250],
          [ 0.1793,  0.3053, -0.0891,  0.4469,  0.4883],
          [-0.0622, -0.1304,  0.1926,  0.2591,  0.4404],
          [ 0.0944,  0.3512,  0.3354,  0.3881,  0.5322]]],


        ...,


        [[[ 0.5209,  0.4986,  0.5187, -0.1027,  0.6966],
          [ 0.2983,  0.7806,  0.4102,  0.0292,  0.0701],
    

tensor([[[[-0.0136, -0.2407, -0.1060,  0.5737, -0.1012],
          [ 0.1682, -0.0440,  0.5148,  0.7115,  0.5687],
          [ 0.5914,  0.3073,  0.7647,  0.4631,  0.5266],
          [ 0.5126,  0.3898,  0.5129,  0.5072,  0.3920],
          [ 0.1739,  0.3780, -0.0565,  0.8841,  0.2094]]],


        [[[ 0.2769,  0.2192,  0.2228,  0.2374,  0.0857],
          [ 0.4258,  0.0474,  0.4744,  0.7028,  0.4913],
          [ 0.5954,  0.5196,  0.7530,  0.5446,  0.3527],
          [ 0.4627,  0.1940,  0.5212, -0.1657,  0.4038],
          [ 0.2904,  0.7572,  0.2598,  0.4926,  0.3253]]],


        [[[ 0.4439,  0.5963,  0.5554,  0.3550,  0.1570],
          [ 0.7325,  0.7591,  0.8513,  0.4449,  0.3250],
          [ 0.1793,  0.3053, -0.0891,  0.4469,  0.4883],
          [-0.0622, -0.1304,  0.1926,  0.2591,  0.4404],
          [ 0.0944,  0.3512,  0.3354,  0.3881,  0.5322]]],


        ...,


        [[[ 0.5209,  0.4986,  0.5187, -0.1027,  0.6966],
          [ 0.2983,  0.7806,  0.4102,  0.0292,  0.0701],
    

tensor([[[[-0.0136, -0.2407, -0.1060,  0.5737, -0.1012],
          [ 0.1682, -0.0440,  0.5148,  0.7115,  0.5687],
          [ 0.5914,  0.3073,  0.7647,  0.4631,  0.5266],
          [ 0.5126,  0.3898,  0.5129,  0.5072,  0.3920],
          [ 0.1739,  0.3780, -0.0565,  0.8841,  0.2094]]],


        [[[ 0.2769,  0.2192,  0.2228,  0.2374,  0.0857],
          [ 0.4258,  0.0474,  0.4744,  0.7028,  0.4913],
          [ 0.5954,  0.5196,  0.7530,  0.5446,  0.3527],
          [ 0.4627,  0.1940,  0.5212, -0.1657,  0.4038],
          [ 0.2904,  0.7572,  0.2598,  0.4926,  0.3253]]],


        [[[ 0.4439,  0.5963,  0.5554,  0.3550,  0.1570],
          [ 0.7325,  0.7591,  0.8513,  0.4449,  0.3250],
          [ 0.1793,  0.3053, -0.0891,  0.4469,  0.4883],
          [-0.0622, -0.1304,  0.1926,  0.2591,  0.4404],
          [ 0.0944,  0.3512,  0.3354,  0.3881,  0.5322]]],


        ...,


        [[[ 0.5209,  0.4986,  0.5187, -0.1027,  0.6966],
          [ 0.2983,  0.7806,  0.4102,  0.0292,  0.0701],
    

tensor([[[[-0.0136, -0.2407, -0.1060,  0.5737, -0.1012],
          [ 0.1682, -0.0440,  0.5148,  0.7115,  0.5687],
          [ 0.5914,  0.3073,  0.7647,  0.4631,  0.5266],
          [ 0.5126,  0.3898,  0.5129,  0.5072,  0.3920],
          [ 0.1739,  0.3780, -0.0565,  0.8841,  0.2094]]],


        [[[ 0.2769,  0.2192,  0.2228,  0.2374,  0.0857],
          [ 0.4258,  0.0474,  0.4744,  0.7028,  0.4913],
          [ 0.5954,  0.5196,  0.7530,  0.5446,  0.3527],
          [ 0.4627,  0.1940,  0.5212, -0.1657,  0.4038],
          [ 0.2904,  0.7572,  0.2598,  0.4926,  0.3253]]],


        [[[ 0.4439,  0.5963,  0.5554,  0.3550,  0.1570],
          [ 0.7325,  0.7591,  0.8513,  0.4449,  0.3250],
          [ 0.1793,  0.3053, -0.0891,  0.4469,  0.4883],
          [-0.0622, -0.1304,  0.1926,  0.2591,  0.4404],
          [ 0.0944,  0.3512,  0.3354,  0.3881,  0.5322]]],


        ...,


        [[[ 0.5209,  0.4986,  0.5187, -0.1027,  0.6966],
          [ 0.2983,  0.7806,  0.4102,  0.0292,  0.0701],
    

...

tensor([[[[-0.0136, -0.2407, -0.1060,  0.5737, -0.1012],
          [ 0.1682, -0.0440,  0.5148,  0.7115,  0.5687],
          [ 0.5914,  0.3073,  0.7647,  0.4631,  0.5266],
          [ 0.5126,  0.3898,  0.5129,  0.5072,  0.3920],
          [ 0.1739,  0.3780, -0.0565,  0.8841,  0.2094]]],


        [[[ 0.2769,  0.2192,  0.2228,  0.2374,  0.0857],
          [ 0.4258,  0.0474,  0.4744,  0.7028,  0.4913],
          [ 0.5954,  0.5196,  0.7530,  0.5446,  0.3527],
          [ 0.4627,  0.1940,  0.5212, -0.1657,  0.4038],
          [ 0.2904,  0.7572,  0.2598,  0.4926,  0.3253]]],


        [[[ 0.4439,  0.5963,  0.5554,  0.3550,  0.1570],
          [ 0.7325,  0.7591,  0.8513,  0.4449,  0.3250],
          [ 0.1793,  0.3053, -0.0891,  0.4469,  0.4883],
          [-0.0622, -0.1304,  0.1926,  0.2591,  0.4404],
          [ 0.0944,  0.3512,  0.3354,  0.3881,  0.5322]]],


        ...,


        [[[ 0.5209,  0.4986,  0.5187, -0.1027,  0.6966],
          [ 0.2983,  0.7806,  0.4102,  0.0292,  0.0701],
    

tensor([[[[-0.0136, -0.2407, -0.1060,  0.5737, -0.1012],
          [ 0.1682, -0.0440,  0.5148,  0.7115,  0.5687],
          [ 0.5914,  0.3073,  0.7647,  0.4631,  0.5266],
          [ 0.5126,  0.3898,  0.5129,  0.5072,  0.3920],
          [ 0.1739,  0.3780, -0.0565,  0.8841,  0.2094]]],


        [[[ 0.2769,  0.2192,  0.2228,  0.2374,  0.0857],
          [ 0.4258,  0.0474,  0.4744,  0.7028,  0.4913],
          [ 0.5954,  0.5196,  0.7530,  0.5446,  0.3527],
          [ 0.4627,  0.1940,  0.5212, -0.1657,  0.4038],
          [ 0.2904,  0.7572,  0.2598,  0.4926,  0.3253]]],


        [[[ 0.4439,  0.5963,  0.5554,  0.3550,  0.1570],
          [ 0.7325,  0.7591,  0.8513,  0.4449,  0.3250],
          [ 0.1793,  0.3053, -0.0891,  0.4469,  0.4883],
          [-0.0622, -0.1304,  0.1926,  0.2591,  0.4404],
          [ 0.0944,  0.3512,  0.3354,  0.3881,  0.5322]]],


        ...,


        [[[ 0.5209,  0.4986,  0.5187, -0.1027,  0.6966],
          [ 0.2983,  0.7806,  0.4102,  0.0292,  0.0701],
    

tensor([[[[-0.0136, -0.2407, -0.1060,  0.5737, -0.1012],
          [ 0.1682, -0.0440,  0.5148,  0.7115,  0.5687],
          [ 0.5914,  0.3073,  0.7647,  0.4631,  0.5266],
          [ 0.5126,  0.3898,  0.5129,  0.5072,  0.3920],
          [ 0.1739,  0.3780, -0.0565,  0.8841,  0.2094]]],


        [[[ 0.2769,  0.2192,  0.2228,  0.2374,  0.0857],
          [ 0.4258,  0.0474,  0.4744,  0.7028,  0.4913],
          [ 0.5954,  0.5196,  0.7530,  0.5446,  0.3527],
          [ 0.4627,  0.1940,  0.5212, -0.1657,  0.4038],
          [ 0.2904,  0.7572,  0.2598,  0.4926,  0.3253]]],


        [[[ 0.4439,  0.5963,  0.5554,  0.3550,  0.1570],
          [ 0.7325,  0.7591,  0.8513,  0.4449,  0.3250],
          [ 0.1793,  0.3053, -0.0891,  0.4469,  0.4883],
          [-0.0622, -0.1304,  0.1926,  0.2591,  0.4404],
          [ 0.0944,  0.3512,  0.3354,  0.3881,  0.5322]]],


        ...,


        [[[ 0.5209,  0.4986,  0.5187, -0.1027,  0.6966],
          [ 0.2983,  0.7806,  0.4102,  0.0292,  0.0701],
    

tensor([[[[-0.0136, -0.2407, -0.1060,  0.5737, -0.1012],
          [ 0.1682, -0.0440,  0.5148,  0.7115,  0.5687],
          [ 0.5914,  0.3073,  0.7647,  0.4631,  0.5266],
          [ 0.5126,  0.3898,  0.5129,  0.5072,  0.3920],
          [ 0.1739,  0.3780, -0.0565,  0.8841,  0.2094]]],


        [[[ 0.2769,  0.2192,  0.2228,  0.2374,  0.0857],
          [ 0.4258,  0.0474,  0.4744,  0.7028,  0.4913],
          [ 0.5954,  0.5196,  0.7530,  0.5446,  0.3527],
          [ 0.4627,  0.1940,  0.5212, -0.1657,  0.4038],
          [ 0.2904,  0.7572,  0.2598,  0.4926,  0.3253]]],


        [[[ 0.4439,  0.5963,  0.5554,  0.3550,  0.1570],
          [ 0.7325,  0.7591,  0.8513,  0.4449,  0.3250],
          [ 0.1793,  0.3053, -0.0891,  0.4469,  0.4883],
          [-0.0622, -0.1304,  0.1926,  0.2591,  0.4404],
          [ 0.0944,  0.3512,  0.3354,  0.3881,  0.5322]]],


        ...,


        [[[ 0.5209,  0.4986,  0.5187, -0.1027,  0.6966],
          [ 0.2983,  0.7806,  0.4102,  0.0292,  0.0701],
    

tensor([[[[-0.0136, -0.2407, -0.1060,  0.5737, -0.1012],
          [ 0.1682, -0.0440,  0.5148,  0.7115,  0.5687],
          [ 0.5914,  0.3073,  0.7647,  0.4631,  0.5266],
          [ 0.5126,  0.3898,  0.5129,  0.5072,  0.3920],
          [ 0.1739,  0.3780, -0.0565,  0.8841,  0.2094]]],


        [[[ 0.2769,  0.2192,  0.2228,  0.2374,  0.0857],
          [ 0.4258,  0.0474,  0.4744,  0.7028,  0.4913],
          [ 0.5954,  0.5196,  0.7530,  0.5446,  0.3527],
          [ 0.4627,  0.1940,  0.5212, -0.1657,  0.4038],
          [ 0.2904,  0.7572,  0.2598,  0.4926,  0.3253]]],


        [[[ 0.4439,  0.5963,  0.5554,  0.3550,  0.1570],
          [ 0.7325,  0.7591,  0.8513,  0.4449,  0.3250],
          [ 0.1793,  0.3053, -0.0891,  0.4469,  0.4883],
          [-0.0622, -0.1304,  0.1926,  0.2591,  0.4404],
          [ 0.0944,  0.3512,  0.3354,  0.3881,  0.5322]]],


        ...,


        [[[ 0.5209,  0.4986,  0.5187, -0.1027,  0.6966],
          [ 0.2983,  0.7806,  0.4102,  0.0292,  0.0701],
    

In [34]:
for epoch in range(opt.epoch, opt.n_epochs):
    for i, batch in enumerate(dataloader):
        # Set model input
        real_Q = Variable(input_A.copy_(batch['A']))#Quantity
        real_P = Variable(input_B.copy_(batch['B']))#Price

        ###### Generators A2B and B2A ######
        optimizer_G.zero_grad()

        # Identity loss
        # G_A2B(B) should equal B if real B is fed
        # 把Price放進 G(Price轉成Quantity)，應該要生成真的Price
        same_P = netG_Q2P(real_P)
        loss_identity_P = criterion_identity(same_P, real_P)*5.0
        # G_B2A(A) should equal A if real A is fed
        same_Q = netG_P2Q(real_Q)
        loss_identity_Q = criterion_identity(same_Q, real_Q)*5.0

        # GAN loss
        fake_P = netG_Q2P(real_Q)
        pred_fake = netD_P(fake_P)
        loss_GAN_Q2P = criterion_GAN(pred_fake, target_real)

        fake_Q = netG_P2Q(real_P)
        pred_fake = netD_Q(fake_Q)
        loss_GAN_P2Q = criterion_GAN(pred_fake, target_real)

        # Cycle loss
        recovered_Q = netG_P2Q(fake_P)
        loss_cycle_QPQ = criterion_cycle(recovered_Q, real_Q)*10.0

        recovered_P = netG_Q2P(fake_Q)
        loss_cycle_PQP = criterion_cycle(recovered_P, real_P)*10.0

        # Total loss
        loss_G = loss_identity_Q + loss_identity_P + loss_GAN_Q2P + loss_GAN_P2Q + loss_cycle_QPQ + loss_cycle_PQP
        loss_G.backward()
        
        optimizer_G.step()
        ###################################

NameError: name 'opt' is not defined